## Uses older PyRosetta bindings



TMalign is an algorithm that can be used to find a matrix of best fit to overlay 2 proteins on one another. It has been encoded into Rosetta and here we use it to align 2 proteins that have different sequences

Note: This does **not** move the ligands if there are any present!!!

In [1]:
#from rosetta import *

import pyrosetta
import  rosetta
pyrosetta.init()

Found rosetta database at: /home/stephanie/anaconda2/envs/DebugPyrosetta/lib/python2.7/site-packages/pyrosetta-4.0-py2.7.egg/database; using it....
PyRosetta-4 2016 [Rosetta 2016 unknown:9ea8e5e15e7c35838a32b8089ca8351ff540888c 2016-12-16 10:52:45 -0500] retrieved from: git@github.com:RosettaCommons/main.git
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.



In [3]:
pose1 = pyrosetta.pose_from_file('pose1.pdb')
pose2 = pyrosetta.pose_from_file('pose2.pdb')

In [4]:
print pose1.residue(1).xyz('CA')
print pose2.residue(1).xyz('CA')

      58.50600000000000       42.69900000000000       28.08300000000000
      61.92500000000000       38.90400000000000       24.13600000000000


Trying to use the TMalign algorithm to do my superposition of fragments/proteins


In [5]:
from rosetta.protocols import hybridization

In [6]:
tm = hybridization.TMalign()
tm.apply(pose1, pose2)       ### note, the proteins have not moved

0L

In [10]:
#from rosetta.core.scoring import create_score_function
sfxn = pyrosetta.create_score_function('talaris2013')
#sfxn = create_score_function('talaris2013')
pm = pyrosetta.PyMolMover()
pm.apply(pose1)
pm.apply(pose2)
#pyrosetta.PyMolMover()
# AddPyMolObserver(pose1, True)
# AddPyMolObserver(pose2, True)
sfxn(pose1)
sfxn(pose2)

-11.708716915711177

In [ ]:
longest = max(pose1.n_residue()+1, pose2.n_residue()+1)
print longest              # need to normalize by one of the lengths of the proteins
shortest = min(pose1.n_residue()+1, pose2.n_residue()+1)
print tm.TMscore(longest)  # yields tmscore of .957 ( or very similar in structure)

In [ ]:
# but the tmalign object doesn't actually move the pose....
print pose1.residue(1).xyz('CA')
print pose2.residue(1).xyz('CA')

## Looking into the Rosetta code base

From /src/protocols/hybridization/HybridizeProtocol.cc, lines 1439 to 1466
demonstrates how to run tmalign and then do the partial threading to actually move the pose

```c++
// get tmalign sequence mapping
		TMalign tm_align;
		std::string seq_pose, seq_ref, aligned;
		int reval = tm_align.apply(pose, ref_pose, residue_list, ref_residue_list);
		if ( reval != 0 ) continue;  // TO DO: remove this domain

		tm_align.alignment2AtomMap(pose, ref_pose, residue_list, ref_residue_list, n_mapped_residues, atom_map);
		tm_align.alignment2strings(seq_pose, seq_ref, aligned);

		using namespace ObjexxFCL::format;
		Size norm_length = residue_list.size() < ref_residue_list.size() ? residue_list.size():ref_residue_list.size();
		TR << "Align domain with TMscore of " << F(8,3,tm_align.TMscore(norm_length)) << std::endl;
		TR << seq_pose << std::endl;
		TR << aligned << std::endl;
		TR << seq_ref << std::endl;

		if ( n_mapped_residues < 6 ) continue;  // TO DO: remove this domain

		// add in ligand residues
		for ( core::Size i=last_protein_residue+1; i<=pose.total_residue(); ++i ) {
			core::Size res_controlling_i = ligres_map[i];
			for ( core::Size iloop=1; iloop<=domains[i_domain].num_loop(); ++iloop ) {
				if ( res_controlling_i < domains[i_domain][iloop].start() || res_controlling_i > domains[i_domain][iloop].stop() ) continue;
				residue_list.push_back(i);
			}
		}

		partial_align(pose, ref_pose, atom_map, residue_list, true, aln_cutoffs, min_coverage);
        ```

So I need to make run this method first:

tm.alignment2AtomMap(pose, ref_pose, residue_list, ref_residue_list, n_mapped_residues, atom_map);

which results below

1413    core::id::AtomID_Map< core::id::AtomID > atom_map;

1414    core::pose::initialize_atomid_map( atom_map, pose, core::id::BOGUS_ATOM_ID )

In [ ]:
from rosetta.core.id import AtomID_Map_T_core_id_AtomID_T
atommap = AtomID_Map_T_core_id_AtomID_T()

In [ ]:
#rosetta.core.pose.initialize_atomid_map_AtomID?
from rosetta.core.pose import initialize_atomid_map
initialize_atomid_map( atommap, pose1 )

In [ ]:
pose1list = utility.vector1_Size()
pose2list = utility.vector1_Size()
for i in xrange(1,shortest):
    pose1list.append(i)
    pose2list.append(i)
print type(pose1list)
#print pose2list

In [ ]:
tm.alignment2AtomMap(pose1,pose2, atommap )

In [ ]:
aln_cutoff = rosetta.utility.vector1_Real()

In [ ]:
for i in [2,1.5,1.0,.5]:
    aln_cutoff.append(i)

In [ ]:
min_coverage = .2

In [ ]:
rosetta.protocols.hybridization.partial_align(pose1,pose2, atommap, True, aln_cutoff, min_coverage)

In [ ]:
# check for movement
print pose1.residue(1).xyz('CA')
print pose2.residue(1).xyz('CA')
sfxn(pose1)

Now let's put that into a function so that we can use it!!!

In [29]:
def tmalign( pose, ref_pose ):
    
    print 'Running tmalign on poses'
#    print 'Starting xyz coords for res1 CA pose and reference pose'
#     print pose.residue(1).xyz('CA')
#     print ref_pose.residue(1).xyz('CA')
    
    tm = rosetta.protocols.hybridization.TMalign()
    tm.apply(pose, ref_pose)
    longest = max(pose.total_residue()+1, ref_pose.total_residue()+1)
    
    print 'TMScore = %s ' %tm.TMscore(longest)
    atommap =  rosetta.core.id.AtomID_Map_core_id_AtomID_t()
    rosetta.core.pose.initialize_atomid_map_AtomID( atommap, pose )
    tm.alignment2AtomMap( pose, ref_pose, atommap )
    
    # some setup for partial thread
    aln_cutoff = rosetta.utility.vector1_double()
    for i in [2,1.5,1.0,.5]:
        aln_cutoff.append(i)
    min_coverage = .2
    rosetta.protocols.hybridization.partial_align(pose1,pose2, atommap, True, aln_cutoff, min_coverage)
    
#     print 'Hopefully these coordinates have changed, use the PyMolMover / Observer to watch in realtime'
#     print pose1.residue(1).xyz('CA')
#     print pose2.residue(1).xyz('CA')

In [30]:
tmalign(pose1, pose2)

Running tmalign on poses
Starting xyz coords for res1 CA pose and reference pose
      58.50600000000000       42.69900000000000       28.08300000000000
      61.92500000000000       38.90400000000000       24.13600000000000
TMScore = 0.957125933343 
Hopefully these coordinates have changed, use the PyMolMover / Observer to watch in realtime
      62.28396568682275       38.67710118134794       24.18507838871280
      61.92500000000000       38.90400000000000       24.13600000000000


In [31]:
pm.apply(pose1)
pm.apply(pose2)